In [24]:
import numpy as np
import pandas as pd
from tensorflow import keras
from keras import backend as K

Using TensorFlow backend.


# Importar dados

As labels dos dados estão definidas tais que:
- Background: 0
- Sinal: 1

In [19]:
x_train = pd.read_hdf("Data/Processed_Data.h5", key='data')
y_train, gen_weights = x_train["Label"].values, x_train["gen_weights"].values
x_train = x_train.drop(["Label", "gen_weights"], axis=1).values

In [23]:
print(y_train.shape, x_train.shape)

(195725,) (195725, 69)


# Build the costum weighted loss function

In [25]:
def weighted_loss(y_true, y_pred, weights):
    # Normalizar pesos
    norm_w = weights / K.sum(weights, axis=-1, keepdims=True)
    
    def crossentropy(y_true, y_pred):
        # Normalizar previsões
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        
        # Evitar NaN e overflows com clipping
        y_pred = K.clip(y_pred, K.epsilon(), 1-K.epsilon())
        
        # Calcular weighted cross_entropy
        result = y_true * norm_w * K.log(y_pred)
        
        return - K.sum(result, axis=-1)
    
    return crossentropy

# Build the model

In [2]:
inputs = keras.Input(shape=(69,))
fc1 = keras.layers.Dense(90, activation='relu')(inputs)
fc2 = keras.layers.Dense(120, activation='relu')(fc1)
fc3 = keras.layers.Dense(80, activation='relu')(fc2)
fc4 = keras.layers.Dense(40, activation='relu')(fc3)
outputs = keras.layers.Dense(2, activation='softmax')(fc4)

model = keras.Model(inputs, outputs)
model.compile(optimizer='adam', loss=weighted_loss(weights))
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 69)]              0         
_________________________________________________________________
dense (Dense)                (None, 90)                6300      
_________________________________________________________________
dense_1 (Dense)              (None, 120)               10920     
_________________________________________________________________
dense_2 (Dense)              (None, 80)                9680      
_________________________________________________________________
dense_3 (Dense)              (None, 40)                3240      
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 82        
Total params: 30,222
Trainable params: 30,222
Non-trainable params: 0
_________________________________________________________